# Librairies

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
from sklearn.linear_model import lasso_path
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 
import seaborn as sns
from mpl_toolkits import mplot3d
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# Lecture des données

In [5]:
PROJECT_PATH = "C:/Users/Utilisateur/Desktop/Projet_interpromo_2k22/ProjectInterPromo"

In [6]:
# Lecture du fichier de donnée
data_clean2 = pd.read_csv(PROJECT_PATH + "/data/donnee_v2_ech_train_transfo.csv")

bool_col = []
for i in data_clean2.columns:
    unique_value = np.unique(data_clean2[i])
    
    if len(unique_value) == 2:
        if 1 in unique_value and 0 in unique_value:
            bool_col.append(i)

y_col = ["TOP_FRAUDE", "TOP_FRAUDE_CARTE", "TOP_FRAUDE_VIREMENT"]

autre_col = list(set(data_clean2.columns) - set(bool_col + y_col + ["ID"]))

sub_df = data_clean2[autre_col]

data_clean2[autre_col] = ((sub_df - sub_df.mean()) / sub_df.std())

# Réduction de dimensions

## ACP

In [7]:
def ACP_3D(df, 
        column_fraude, 
        axes,#[num_axe1 , num_axe2]
        Data_path
       ):
    
    quali_col = []
    for i, name in zip(df.dtypes, df.columns):
        if i == "object":
            quali_col.append(name)

    data_quanti = df.drop(columns=quali_col)

    pca = PCA()
    X_pca=pca.fit_transform(data_quanti.drop(columns=column_fraude))
    
    df_graph_NF = pd.DataFrame({'X' :X_pca[:,axes[0]][data_quanti[column_fraude]==0] ,
                            'Y' :X_pca[:,axes[1]][data_quanti[column_fraude]==0],
                            'Z':X_pca[:,axes[2]][data_quanti[column_fraude]==0],
                            'F/NF': 'Non Fraude'})
    
    df_graph_F = pd.DataFrame({'X' :X_pca[:,axes[0]][data_quanti[column_fraude]==1] ,
                            'Y' :X_pca[:,axes[1]][data_quanti[column_fraude]==1],
                            'Z':X_pca[:,axes[2]][data_quanti[column_fraude]==1],
                            'F/NF': 'Fraude'})
    
    df_graph = pd.concat([df_graph_NF, df_graph_F])
    df_graph.to_csv(Data_path+'/G2_ACP_3D.csv', index= False, sep=';')
    
    return(X_pca[:,axes[0:10]])

ACP_3D(data_clean2, "TOP_FRAUDE", [0,1,2],PROJECT_PATH+'/dev/visualisation/nessrine')

MemoryError: Unable to allocate 2.47 GiB for an array with shape (764471, 433) and data type float64

## LDA

In [ ]:
def LDA (df, 
         column_fraude,
         alpha, #list(alpha0 , alpha1)
         data_path
        ):
    # enlever les variables qualitatives.
    quali_col = []
    for i, name in zip(df.dtypes, df.columns):
        if i == "object":
            quali_col.append(name)

    data_quanti = df.drop(columns=quali_col)
    X , Y = data_quanti.drop(['TOP_FRAUDE', 'TOP_FRAUDE_CARTE', 'TOP_FRAUDE_VIREMENT'],1) , column_fraude
    clf = LinearDiscriminantAnalysis()
    clf.fit(X , Y)
    importances = clf.coef_
    importances = importances.reshape(len(X.columns) , 1)

    d = dict(zip(data_quanti.columns , importances))
    d_tri = sorted(d.items(), key=lambda t: np.abs(t[1]))
    
    plt.scatter(data_quanti[data_quanti["TOP_FRAUDE"]==0][d_tri[-1][0]], data_quanti[data_quanti["TOP_FRAUDE"]==0][d_tri[-2][0]], alpha=alpha[0], c="orange", label="Not Fraude")
    plt.scatter(data_quanti[data_quanti["TOP_FRAUDE"]==1][d_tri[-1][0]], data_quanti[data_quanti["TOP_FRAUDE"]==1][d_tri[-2][0]], alpha=alpha[1], c="blue", label="Fraude")
    plt.legend()
    plt.title('Fraudes sur les 2 meilleurs axes')
    plt.show()
    
    df_final = df[[d_tri[-1][0],d_tri[-2][0], d_tri[-3][0]]]
    df_final['Fraude'] = column_fraude
    df_final.columns = ['X', 'Y', 'Z', 'F/NF']
    df_final.to_csv(data_path+'/G2_LDA.csv')
    return d_tri[-10:]

LDA(d, df['TOP_FRAUDE'], [0.1, 0.5], PROJECT_PATH+'/dev/visualisation/nessrine')

## TSNe

In [ ]:
def T_SNE(
    df,
    column_fraude,
    n_composant,  
    alphas,
    data_path
    ):

    #preparation des entrées du modèles
    X = df.drop(['TOP_FRAUDE', 'TOP_FRAUDE_CARTE', 'TOP_FRAUDE_VIREMENT'],1)
    Y = df[str(column_fraude)]
    tsne = TSNE(n_components = int(n_composant), verbose=1, random_state=123)
    x_tsne = tsne.fit_transform(X) 
    resultat = pd.DataFrame()
    resultat["y"] = Y
    resultat["comp-1"] = x_tsne[:,0]
    resultat["comp-2"] = x_tsne[:,1]

    plt.scatter(resultat["comp-1"][data_quanti[column_fraude]==0], resultat["comp-2"][data_quanti[column_fraude]==0], alpha=alphas[0], c="orange", label="Not Fraude")
    plt.scatter(resultat["comp-1"][data_quanti[column_fraude]==1], resultat["comp-2"][data_quanti[column_fraude]==1], alpha=alphas[1], c="b", label="Fraude")
    plt.legend()
    plt.show()
    
    d = {'coordonnées composante 1' : resultat['comp-1'], 'coordonnées composante 2': resultat["comp-2"]}
    df_final = pd.DataFrame(d)
    df_final.to_csv(data_path + '/G2_T-SNE.csv')
    return x_tsne

T_SNE(df, 'TOP_FRAUDE',3, [0.1, 0.5], PROJECT_PATH+'/dev/visualisation/nessrine')